In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.stats import multivariate_normal as mv_norm





############################# BLR ######################################
class LinearBayes(object):
 
  def __init__(self, a_m0, m_s0, beta):
   self.prior = mv_norm(mean = a_m0, cov = m_s0)
   self.v_m0 = a_m0.reshape(a_m0.shape+(1,))
   self.m_s0 = m_s0
   self.beta = beta

   self.v_mN = self.v_m0
   self.m_sN = self.m_s0
   self.posterior = self.prior
  
  def set_posterior(self, a_x, a_y):
   self.m_sN = np.linalg.inv(np.linalg.inv(self.m_s0)+self.beta*a_x.T.dot(a_x))
   self.v_mN = self.m_sN.dot(np.linalg.inv(self.m_s0).dot(self.v_m0)+self.beta*a_x.T.dot(a_y))
   self.posterior = mv_norm(mean=self.v_mN.flatten(), cov=self.m_sN)
   return self.v_mN

def Gaussian_basis(x1, x2, s1, s2, mu_i, mu_j):
  return math.exp(-(x1-mu_i)**2/(2*(s1**2))-(x2-mu_j)**2/(2*(s2**2))) 
 
BLR_mse = [] 
BLR_m = []

test_y = np.asarray(test_y).reshape(test_y.shape+(1,))

for O1 in range(2,8):                                                                               #令 O1 為 2 到 8
    for O2 in range(2,8):                                                                           #令 O2 為 2 到 8       
        O1_i = [i for i in range(1,O1+1)]                                                           #設定 i 的範圍                      
        O2_j = [i for i in range(1,O2+1)]                                                           #設定 j 的範圍            
        P = O1 * O2                                                                                 # P = O1 * O2
        Phi_tr = np.zeros((train_x.shape[0], P+2))
        s1_tr = (max(train_df['GRE Scores'])-min(train_df['GRE Scores'])) / (O1-1)                  #定義 s1 (可知 O1 不能為 1)
        s2_tr = (max(train_df['TOFEL Scores'])-min(train_df['TOFEL Scores'])) / (O2-1)              #定義 s2 (可知 O2 不能為 1)
        
        for i in O1_i:                                                                              # 1 =< i =< O1
            for j in O2_j:                                                                          # 1 =< j =< O2
                k = O2 * (i - 1) + j                                                                #定義 k (即 1 到 P)
                mu_i_tr = s1_tr * (i-1) + min(train_df['GRE Scores'])                               #定義 mu_i
                mu_j_tr = s2_tr * (j-1) + min(train_df['TOFEL Scores'])                             #定義 mu_j
               
                for n in range(train_df.shape[0]):                                                  #分別取出第 n 筆資料做運算 (train 的筆數)
                    x1_tr = train_df['GRE Scores'][n]                                               #x1 的第 n 筆資料
                    x2_tr = train_df['TOFEL Scores'][n]                                             #x2 的第 n 筆資料
                    Phi_tr[n][k-1] = Gaussian_basis(x1_tr, x2_tr, s1_tr, s2_tr, mu_i_tr, mu_j_tr)
                    Phi_tr[n][(P+1)-1] = train_df['Research Exp.'][n]                               #x3 存於 P+1
                    Phi_tr[n][(P+2)-1] = 1                                                          #bias 存於 P+2
        
        
        Phi_ts = np.zeros((test_x.shape[0], P+2))
        s1_ts = (max(test_df['GRE Scores'])-min(test_df['GRE Scores'])) / (O1-1)
        s2_ts = (max(test_df['TOFEL Scores'])-min(test_df['TOFEL Scores'])) / (O2-1)
        
        for i in O1_i:
            for j in O2_j: 
                k = O2 * (i - 1) + j 
                mu_i_ts = s1_ts * (i-1) + min(test_df['GRE Scores'])
                mu_j_ts = s2_ts * (j-1) + min(test_df['TOFEL Scores'])   
                
                for n in range(test_df.shape[0]):
                    x1_ts = test_df['GRE Scores'][n]
                    x2_ts = test_df['TOFEL Scores'][n]
                    Phi_ts[n][k-1] = Gaussian_basis(x1_ts, x2_ts, s1_ts, s2_ts, mu_i_ts, mu_j_ts)
                    Phi_ts[n][(P+1)-1] = test_df['Research Exp.'][n]                  
                    Phi_ts[n][(P+2)-1] = 1
                    
noise_sigma = 1.1 #這裡沒包再回圈裡
beta = 1/noise_sigma**2

x_real = Phi_tr
y_real = pd.DataFrame(train_y).to_numpy()

a_m0 = np.array([0. for i in range(len(x_real[1]))])
alpha = 2
m_s0 = 1/alpha*np.identity(len(x_real[1]))

linbayes = LinearBayes(a_m0, m_s0, beta)
v_mN = linbayes.set_posterior(x_real, y_real)
BLR_yhat = Phi_ts.dot(v_mN)                                                                   #計算 yhat
BLR_square_error = (test_y - BLR_yhat)**2                                                     #計算 square error
BLR_mse.append([np.mean(BLR_square_error), O1, O2])                                           #計算及儲存 mean square error
BLR_m.append([np.mean(BLR_square_error)])

print('Best choice of [O1, O2] =', min(BLR_mse)[1:])

print('MSE of BLR =',np.min(BLR_m) )

Best choice of [O1, O2] = [7, 7]
MSE of BLR = 0.0053592788810285265


In [ ]:
#指定格式，她改過的，這應該才是對的，上面的可能有錯

import numpy as np
import pandas as pd
import math
import scipy.stats
from scipy.stats import multivariate_normal as mv_norm
import argparse


# do not change the name of this function
def BLR(train_data, test_data_feature, O1=5, O2=5):  # remember to set best choice O1 and O2 as default
    '''
    output: ndarray with size (length of test_data, )
    '''


    return y_BLRprediction 


# do not change the name of this function
def MLR(train_data, test_data_feature, O1=5, O2=5):  # remember to set best choice O1 and O2 as default
    '''
    output: ndarray with size (length of test_data, )
    '''


    return y_MLRprediction


def CalMSE(data, prediction):

    squared_error = (data - prediction) ** 2
    sum_squared_error = np.sum(squared_error)
    mean__squared_error = sum_squared_error/prediction.shape[0]

    return mean__squared_error


def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('-O1', '--O_1', type=int, default=5)
    parser.add_argument('-O2', '--O_2', type=int, default=5)
    args = parser.parse_args()
    O_1 = args.O_1
    O_2 = args.O_2
    
    data_train = pd.read_csv('Training_set.csv', header=None).to_numpy()
    data_test = pd.read_csv('Validation_set.csv', header=None).to_numpy()
    data_test_feature = data_test[:, :3]
    data_test_label = data_test[:, 3]
    
    predict_BLR = BLR(data_train, data_test_feature, O1=O_1, O2=O_2)
    predict_MLR = MLR(data_train, data_test_feature, O1=O_1, O2=O_2)

    print('MSE of BLR = {e1}, MSE of MLR= {e2}.'.format(e1=CalMSE(predict_BLR, data_test_label), e2=CalMSE(predict_MLR, data_test_label)))


if __name__ == '__main__':
    main()